# 🤺 Fence demo

This notebook demonstrates the use of various classes in this package. The core intent of this package is to provide a framework for interacting with language models in a way that is modular, extensible, and easy to use. The core classes are:
- `PromptTemplate`: A template for a prompt that can be rendered with a dictionary of variables (or using keywords). It wraps around jinja2.Template, but adds some additional functionality.
- `LLM`: A wrapper around a language model. 
- `Link`: Atomic LLM interaction wrapper. Takes in a PromptTemplate and a language model. It can be invoked with a dictionary of variables, and will return a dictionary of variables.
- `TransformationLink`: A wrapper around a `function` that takes a dictionary of variables and returns a dictionary of variables. It can be invoked with a dictionary of variables, and will return a dictionary of variables.
- `Chain`: A collection of links that are invoked in the right order based on the input and output keys for each Link.
- `LinearChain`: A sequence of links that are invoked in the order they are passed in.

Through these classes, we can interact with LLMs in varying degrees of complexity:
1. Just call the model directly
2. Use a `PromptTemplate` for reusability and abstraction
3. Use a `Link` for atomic LLM interaction
4. Use a `Chain` to execute a collection of `Links`

Below, we'll go through each of these levels of complexity and provide examples.

**Note** This notebook assumes you have access to AWS Bedrock, as we use Bedrock's Claude-instant model to fuel our LLM interactions.

In [1]:
%load_ext autoreload
from fence import Link, LLM, ClaudeInstantLLM, TransformationLink, Chain, LinearChain, PromptTemplate

## ⚙️ Setting up

In [2]:
# Get our model
claude = ClaudeInstantLLM(source='demo_notebook')

### 🪨 Level 1 - Just call the damn thing

In [3]:
# Use the invoke method to call the model
claude.invoke('Why is the sky blue?')

" The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue light there is, the more blue the sky appears."

In [4]:
# Just call the damn thing
claude('Why is the sky blue?')

" The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from every direction in the sky. The more scattered blue light entering our eyes makes the sky appear blue."

### 🔨 Level 2 - Use a PromptTemplate

In [5]:
# Initialize a prompt template
prompt_template = PromptTemplate('Why is the sky {{color}}?', input_variables=['color'])
print(prompt_template)

PromptTemplate(source=Why is the sky {{color}}?, input_variables=['color'])


In [6]:
# Render it with a dictionary
print(prompt_template.render({'color': 'blue'}))

# Render it with keyword arguments
print(prompt_template.render(color='red'))

# Input dict takes precedence over keyword arguments
print(prompt_template.render(input_dict={'color': 'blue'}, color='red'))

Why is the sky blue?
Why is the sky red?
Why is the sky blue?


In [7]:
# You can concatenate prompt templates, input variables are merged
prompt_template_sky = PromptTemplate('Why is the sky {{color}}?', input_variables=['color'])
prompt_template_grass = PromptTemplate('Why is the grass {{color}}?', input_variables=['color'])
prompt_template_dress = PromptTemplate('I like a dress with {{pattern}}.', input_variables=['pattern'])
combined_prompt_template = prompt_template_sky + prompt_template_grass + prompt_template_dress
print(combined_prompt_template)
print(combined_prompt_template.render({'color': 'blue', 'pattern': 'polka dots'}))

PromptTemplate(source=Why is the sky {{color}}? Why is the grass {{color}}? I like a dress with {{pattern}}., input_variables=['pattern', 'color'])
Why is the sky blue? Why is the grass blue? I like a dress with polka dots.


In [8]:
# You can customize the separator
base_template = PromptTemplate('Why is the sky {{color}}?', input_variables=['color'], separator=' FUNKY TOWN ')
additional_template = PromptTemplate('Why is the grass {{color}}?', input_variables=['color'])
print((base_template + additional_template).render({'color': 'blue'}))

Why is the sky blue? FUNKY TOWN Why is the grass blue?


### 🧠 Level 3 - Use Links (lol)

What are Links? In this context, they represent atomic components of LLM interaction. That means they should be able to be strung together to form a Chain, although they can be used independently as well.

In [9]:
# The simplest link is the Link class, which just takes a prompt template and a model
link = Link(template=prompt_template, llm=claude)
print(link)

Link: <['color']> -> <state>


In [10]:
# Invoke it using the input_dict, NOT keyword arguments
link(input_dict={'color': 'blue'}) # Or, equivalently, link.run(input_dict={'color': 'blue'})

[20:07:29] INFO     Executing  Link                                                                                                                 ]8;id=253459;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py\links.py]8;;\:]8;id=166077;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py#198\198]8;;\

{'state': " The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from every direction in the sky. The more scattered blue light entering our eyes makes the sky appear blue."}

In [11]:
# By default, output is stored under 'state'. You can get a copy (e.g., for inspection of intermediate results) by passing a different output key
link = Link(template=prompt_template, llm=claude, output_key='intermediate')
link.run(input_dict={'color': 'blue'})

[20:07:30] INFO     Executing  Link                                                                                                                 ]8;id=311584;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py\links.py]8;;\:]8;id=281501;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py#198\198]8;;\

{'state': " The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue light there is, the more blue the sky appears.",
 'intermediate': " The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue light there is, the more b

In [12]:
# You can name your links for easier debugging in logs
link = Link(template=prompt_template, llm=claude, name='sky')
link.run(input_dict={'color': 'blue'})

[20:07:32] INFO     Executing <sky> Link                                                                                                            ]8;id=373268;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py\links.py]8;;\:]8;id=834889;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py#198\198]8;;\

{'state': " The sky appears blue because of Rayleigh scattering. This is when sunlight interacts with molecules in Earth's atmosphere, primarily nitrogen and oxygen. The short wavelengths of light, such as blue and violet, scatter more than the longer wavelengths such as red and infrared. This scattering process causes the blue wavelengths to be scattered all throughout the atmosphere and reach our eyes from all angles of the sky. The more scattered blue wavelengths predominate, giving the sky its blue color."}

In [13]:
# You can also build TransformationLinks, which take a function that transforms any input_dict into a specific output
def concatenate(input: dict):
    return f"{input['X']} and {input['Y']}"

concat_link = TransformationLink(
    input_keys=["X", "Y"], function=concatenate, output_key="C"
)

concat_link.run(input_dict={"X": "Hello", "Y": "World"})

[20:07:33] INFO     Executing  Link                                                                                                                 ]8;id=345970;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py\links.py]8;;\:]8;id=714554;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py#130\130]8;;\

{'state': 'Hello and World', 'C': 'Hello and World'}

### 🚀 Level 4 - Use Chains (lol again)

In [14]:
# You can also build Chains, which are just a sequence of links. There are two types of chains: LinearChain and Chain. 
# LinearChain is a sequence of links, while Chain is a collection of links that are invoked in the right order based on the input and output keys for each Link.

# This is a LinearChain #
#########################

# Build some links
link_opposite = Link(
    template=PromptTemplate(
        "What's the opposite of {{A}}? Reply with a few words max.", ["A"]
    ),
    name = 'opposite',
    output_key="X",
)
link_superlative = Link(
    template=PromptTemplate(
        "What's a synonym for {{B}}. Reply with one word.", ["B"],
    ),
    name='superlative',
    output_key="Y",
)
link_poem = Link(
    template=PromptTemplate(
        "Write a poem about {{state}}. Return only the poem, beginning with the title.", ["state"]
    ),
    name='poem',
    output_key="Z",
)

# Now build a LinearChain
linear_chain = LinearChain(llm=claude, links=[link_opposite, link_superlative, concat_link, link_poem])

# Run it
result = linear_chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})

# Get the output
print(result['state'])

           INFO     Executing <opposite> Link                                                                                                       ]8;id=148234;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py\links.py]8;;\:]8;id=841677;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py#198\198]8;;\

[20:07:34] INFO     🔑 State keys: ['A', 'B', 'state', 'X'] (added: ['state', 'X'], removed: [])                                                   ]8;id=780067;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/chains.py\chains.py]8;;\:]8;id=622152;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/chains.py#250\250]8;;\

           INFO     Executing <superlative> Link                                                                                                    ]8;id=42144;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py\links.py]8;;\:]8;id=202486;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py#198\198]8;;\

           INFO     🔑 State keys: ['A', 'B', 'state', 'X', 'Y'] (added: ['Y'], removed: [])                                                       ]8;id=432207;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/chains.py\chains.py]8;;\:]8;id=316785;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/chains.py#250\250]8;;\

           INFO     Executing  Link                                                                                                                 ]8;id=303018;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py\links.py]8;;\:]8;id=354089;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py#130\130]8;;\

           INFO     🔑 State keys: ['A', 'B', 'state', 'X', 'Y', 'C'] (added: ['C'], removed: [])                                                  ]8;id=589553;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/chains.py\chains.py]8;;\:]8;id=626914;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/chains.py#250\250]8;;\

           INFO     Executing <poem> Link                                                                                                           ]8;id=167569;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py\links.py]8;;\:]8;id=941477;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py#198\198]8;;\

[20:07:39] INFO     🔑 State keys: ['A', 'B', 'state', 'X', 'Y', 'C', 'Z'] (added: ['Z'], removed: [])                                             ]8;id=777230;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/chains.py\chains.py]8;;\:]8;id=865737;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/chains.py#250\250]8;;\

 The Criminal Optimist

I've broken many laws without regret  
My actions others surely won't forget
Yet still I dream that things will turn out right
Hoping the judge will go easy in her might  

Behind these bars my mind begins to soar  
Imagining the life that's in store  
Once I've paid my debt to society
The future's full of hope and possibility

My heart says this is just a minor slip  
That I'll learn from mistakes, then get my grip
So though the evidence points me to this cell
This criminal stays stubbornly optimistic as well


In [15]:
# A LinearChain will take the presence of the 'state' key into account when invoking the next link.
# A Chain will not. However, it has an extra 'feature' in the form of topological sorting. As long as a graph of links can be
# extracted from the chain, and the input keys (that are not generated in the chain) are given, the chain will invoke the links in the right order.

# This is a Chain #
###################
link_a = Link(
    template=PromptTemplate(
        "Capitalize this word: {{A}}. Only respond with the capitalized version", ["A"]
    ),
    name = 'opposite',
    output_key="X",
)
link_b = Link(
    template=PromptTemplate(
        "What's a synonym for {{B}}. Reply with one word.", ["B"],
    ),
    name='superlative',
    output_key="Y",
)
link_c = Link(
    template=PromptTemplate(
        "Combine {{X}} and {{Y}} and {{C}} in a meaningful sentence.", ["X", "Y", "C"]
    ),
    name='sentence',
    output_key="Z",
)
chain = Chain(llm=claude, links=[link_c, link_a, link_b]) # Note that we can pass the links in any order

# This is the sorted graph of links
chain._topological_sort()

[Link: superlative <['B']> -> <Y>,
 Link: opposite <['A']> -> <X>,
 Link: sentence <['X', 'Y', 'C']> -> <Z>]

In [16]:
# Now we can run it
try:
    result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})
except Exception as e:
    print(e)

The following input keys are required: {'B', 'C', 'A'}. Missing: {'C'}


In [17]:
# Woops, forgot something! There's no link that generates the 'C' key. We can pass it in though.
result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful", "C": "a dog"})
print(result['state'])

           INFO     Executing <superlative> Link                                                                                                    ]8;id=96439;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py\links.py]8;;\:]8;id=496934;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py#198\198]8;;\

           INFO     Executing <opposite> Link                                                                                                       ]8;id=525482;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py\links.py]8;;\:]8;id=833952;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py#198\198]8;;\

           INFO     Executing <sentence> Link                                                                                                       ]8;id=91372;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py\links.py]8;;\:]8;id=89168;file:///Users/wouter.durnez/Documents/Repositories/showpad_personal/fence/fence/src/llm/links.py#198\198]8;;\

 Here is a sentence combining those elements:

The optimistic police officer's dog always believed in him and helped brighten his day, even after difficult calls.


In [18]:
# Cycles are not allowed
link_up = Link(
    template=PromptTemplate(
        "Capitalize this word: {{up}}. Only respond with the capitalized version", ["up"]
    ),
    name = 'up',
    output_key="down",
)
link_b = Link(
    template=PromptTemplate(
        "What's a synonym for {{down}}. Reply with one word.", ["down"],
    ),
    name='down',
    output_key="up",
)
chain = Chain(llm=claude, links=[link_up, link_b])
try:
    chain._topological_sort()
except Exception as e:
    print(e)

Cycle detected in the dependency graph.
